In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
# Hyperparameters
batch_size = 64
learning_rate = 1e-3
num_epochs = 50

# Transform: Normalize and Flatten Images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load MNIST Dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


<a id="4"></a> <br>
### Artificial Neural Network (ANN)

In [15]:
# Create ANN Model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()

        self.fc1 = nn.Linear(28 * 28, 128) 
        self.att1 = nn.Linear(128, 128)  

        self.fc2 = nn.Linear(128, 64)
        self.att2 = nn.Linear(64, 64)  

        self.fc3 = nn.Linear(64, 32)
        self.att3 = nn.Linear(32, 32)  

        self.fc4 = nn.Linear(32, 10)  

    def forward(self, x):
        # First layer
        out1 = torch.relu(self.fc1(x))
        att1 = torch.softmax(self.att1(out1), dim=1)  # Compute attention
        out1 = out1 * att1  # Apply attention

        # Second layer
        out2 = torch.relu(self.fc2(out1))
        att2 = torch.softmax(self.att2(out2), dim=1)
        out2 = out2 * att2

        # Third layer
        out3 = torch.relu(self.fc3(out2))
        att3 = torch.softmax(self.att3(out3), dim=1)
        out3 = out3 * att3

        # Output layer
        out4 = self.fc4(out3)  # No Softmax here! (CrossEntropyLoss expects raw logits)

        return out4


model = ANNModel().to(device)
error = nn.CrossEntropyLoss()
# learning_rate = 0.02
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [16]:
# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/50], Loss: 0.0014, Acc: 47.9400%
Epoch [1/50], Loss: 0.0007, Acc: 68.9000%
Epoch [2/50], Loss: 0.0006, Acc: 73.3000%
Epoch [3/50], Loss: 0.0003, Acc: 79.0000%
Epoch [4/50], Loss: 0.0004, Acc: 83.4700%
Epoch [5/50], Loss: 0.0005, Acc: 81.4600%
Epoch [6/50], Loss: 0.0001, Acc: 85.1200%
Epoch [7/50], Loss: 0.0002, Acc: 90.2000%
Epoch [8/50], Loss: 0.0003, Acc: 92.6000%
Epoch [9/50], Loss: 0.0005, Acc: 94.2300%
Epoch [10/50], Loss: 0.0001, Acc: 94.4600%
Epoch [11/50], Loss: 0.0001, Acc: 95.1200%
Epoch [12/50], Loss: 0.0000, Acc: 94.8100%
Epoch [13/50], Loss: 0.0000, Acc: 95.2900%
Epoch [14/50], Loss: 0.0001, Acc: 95.2700%
Epoch [15/50], Loss: 0.0007, Acc: 95.6500%
Epoch [16/50], Loss: 0.0000, Acc: 95.7900%
Epoch [17/50], Loss: 0.0001, Acc: 96.1000%
Epoch [18/50], Loss: 0.0001, Acc: 94.3900%
Epoch [19/50], Loss: 0.0000, Acc: 95.7800%
Epoch [20/50], Loss: 0.0003, Acc: 95.2100%
Epoch [21/50], Loss: 0.0001, Acc: 96.3300%
Epoch [22/50], Loss: 0.0001, Acc: 96.2300%
Epoch [23/50], Loss: 

In [18]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("ANN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("ANN: Accuracy vs Number of iteration")
# plt.show()

<a id="5"></a> <br>
### Convolutional Neural Network (CNN)
- CNN is well adapted to classify images.
- You can learn CNN basics: https://www.kaggle.com/kanncaa1/convolutional-neural-network-cnn-tutorial
- **Steps of CNN:**
    1. Import Libraries
    1. Prepare Dataset
        - Totally same with previous parts.
        - We use same dataset so we only need train_loader and test_loader. 
    1. Convolutional layer: 
        - Create feature maps with filters(kernels).
        - Padding: After applying filter, dimensions of original image decreases. However, we want to preserve as much as information about the original image. We can apply padding to increase dimension of feature map after convolutional layer.
        - We use 2 convolutional layer.
        - Number of feature map is out_channels = 16
        - Filter(kernel) size is 5*5
    1. Pooling layer: 
        - Prepares a condensed feature map from output of convolutional layer(feature map) 
        - 2 pooling layer that we will use max pooling.
        - Pooling size is 2*2
    1. Flattening: Flats the features map
    1. Fully Connected Layer: 
        - Artificial Neural Network that we learnt at previous part.
        - Or it can be only linear like logistic regression but at the end there is always softmax function.
        - We will not use activation function in fully connected layer.
        - You can think that our fully connected layer is logistic regression.
        - We combine convolutional part and logistic regression to create our CNN model.
    1. Instantiate Model Class
        - create model
    1. Instantiate Loss
        - Cross entropy loss
        - It also has softmax(logistic function) in it.
    1. Instantiate Optimizer
        - SGD Optimizer
    1. Traning the Model
    1. Prediction
- As a result, as you can see from plot, while loss decreasing, accuracy is increasing and our model is learning(training). 
- Thanks to convolutional layer, model learnt better and accuracy(almost 98%) is better than accuracy of ANN. Actually while tuning hyperparameters, increase in iteration and expanding convolutional neural network can increase accuracy but it takes too much running time that we do not want at kaggle.   
        

In [19]:
# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out


model = CNNModel().to(device)
error = nn.CrossEntropyLoss()
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device).view(-1, 1, 28, 28)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device).view(-1, 1, 28, 28)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

In [29]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("CNN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("CNN: Accuracy vs Number of iteration")
# plt.show()